# [Deploy and Run] Dploy the Trained Custom Speech Models
This sample demonstrates how to deploy and run the trained Custom Speech models calling REST API. 

> ✨ ***Note*** <br>
> Except for batch transcription, you must deploy a custom endpoint to use a custom speech model. You can deploy an endpoint for a base or custom model, and then update the endpoint later to use a better trained model. Endpoints used by F0 Speech resources are deleted after seven days.

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

```bash
pip install -r requirements.txt
```

Create an .env file based on the .env-sample file. Copy the new .env file to the folder containing your notebook and update the variables.

## Setup the environment

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import json
from openai import AzureOpenAI
import requests
from dotenv import load_dotenv
from utils.common import *

load_dotenv()

speech_key = os.getenv("AZURE_AI_SPEECH_API_KEY")
speech_region = os.getenv("AZURE_AI_SPEECH_REGION")

# Get the project and custom model IDs from the previous notebook
project_id = ""
custom_model_with_plain_id = ""
custom_model_with_acoustic_id = ""
%store -r project_id
%store -r custom_model_with_plain_id
%store -r custom_model_with_acoustic_id

try:
    project_id, custom_model_with_plain_id, custom_model_with_acoustic_id
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the previous notebook again.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print(project_id, custom_model_with_plain_id, custom_model_with_acoustic_id)

8b0eb25b-38f1-4f7b-8cfe-641753cd9cd1 762281d5-7387-45cf-8a1b-13f2670b15f3 d44c77c4-08b0-4851-aa97-a1417aa3ddc4


## 1. Deploy a trained speech model
- In order to get more information about deploying the custom models, check this document link below
- https://learn.microsoft.com/en-us/azure/ai-services/speech-service/how-to-custom-speech-deploy-model?pivots=rest-api

In [3]:
import requests
import time
import json

# Base URL for the Speech Services REST API
base_url = f'https://{speech_region}.api.cognitive.microsoft.com/speechtotext'

# Headers for authentication
headers = {
    'Ocp-Apim-Subscription-Key': speech_key,
    'Content-Type': 'application/json'
}

### Check the custom speech model ids to deploy

In [ ]:
# Set the base, custom model id to evaluate
base_model_id = "your base model id"  ##check the model id from the train a new model ()
print(custom_model_with_plain_id, custom_model_with_acoustic_id)

# Set the model id to deploy
deploy_model_id = custom_model_with_plain_id

762281d5-7387-45cf-8a1b-13f2670b15f3 d44c77c4-08b0-4851-aa97-a1417aa3ddc4


## Create an endpoint to deploy the custom speech model

In [14]:
display_name = "My custom model endpoint"
description = "Custom model endpoint to deloy"
locale = "vi-VN"

endpoint_id = create_endpoint(base_url, headers, project_id, deploy_model_id, display_name, description, locale)

https://swedencentral.api.cognitive.microsoft.com/speechtotext/v3.2/endpoints
8b0eb25b-38f1-4f7b-8cfe-641753cd9cd1 762281d5-7387-45cf-8a1b-13f2670b15f3
{'model': {'self': 'https://swedencentral.api.cognitive.microsoft.com/speechtotext/v3.2/models/762281d5-7387-45cf-8a1b-13f2670b15f3'}, 'project': {'self': 'https://swedencentral.api.cognitive.microsoft.com/speechtotext/v3.2/projects/8b0eb25b-38f1-4f7b-8cfe-641753cd9cd1'}, 'displayName': 'My custom model endpoint', 'description': 'Custom model endpoint to deloy', 'locale': 'vi-VN'}
Create Endpoint and deploy job finished with ID: 4b91679e-8605-4a30-8496-7b16df2df866


In [15]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_endpoint_status(endpoint_id):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = get_endpoint_status(base_url, headers, endpoint_id)
        if status == "NotStarted":
            pbar.update(1)
        while status != "Succeeded" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = get_endpoint_status(base_url, headers, endpoint_id)
        while(pbar.n < 3):
            pbar.update(1)
        print("Endpoint Creation Completed")

In [16]:
monitor_endpoint_status(endpoint_id)

Running Status:  33%|███▎      | 1/3 [00:01<00:02,  1.05s/step]

Current Status: Running


Running Status:  67%|██████▋   | 2/3 [00:12<00:06,  6.93s/step]

Current Status: Running
Current Status: Running


Running Status: 100%|██████████| 3/3 [00:34<00:00, 11.41s/step]

Endpoint Creation Completed


## 2. Test the deployed custom model 

- Test your deployed custom model by Python SDK. In order to set the endpoint, you need to get the endpoint id from the previous step.

In [20]:
def speech_recognition_from_file_with_endpoint(file_path: str, lang:str, endoint_id: str):
    # Check the Parameters - https://learn.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.speechconfig?view=azure-python
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region, speech_recognition_language=lang)
    speech_config.endpoint_id = endoint_id
    audio_config = speechsdk.AudioConfig(filename=file_path)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    speech_recognition_result = speech_recognizer.recognize_once_async().get()
    return speech_recognition_result.text

### Get the sorted wav files from the dataset folder

In [18]:
import os
from IPython.display import Audio, display

output_folder = 'synthetic_data'
files = os.listdir(output_folder)
wav_files = [file for file in files if file.endswith('.wav')]

# Sort wav_files by 'no' in ascending order
wav_files.sort(key=lambda x: int(x.split('_')[0]))
wav_files

['1_vi-VN_20241110220608.wav',
 '2_vi-VN_20241110220612.wav',
 '3_vi-VN_20241110220616.wav',
 '4_vi-VN_20241110220620.wav',
 '5_vi-VN_20241110220624.wav',
 '6_vi-VN_20241110220629.wav',
 '7_vi-VN_20241110220632.wav',
 '8_vi-VN_20241110220636.wav',
 '9_vi-VN_20241110220640.wav',
 '10_vi-VN_20241110220644.wav',
 '11_vi-VN_20241110220648.wav',
 '12_vi-VN_20241110220652.wav',
 '13_vi-VN_20241110220655.wav',
 '14_vi-VN_20241110220659.wav',
 '15_vi-VN_20241110220704.wav',
 '16_vi-VN_20241110220708.wav',
 '17_vi-VN_20241110220711.wav',
 '18_vi-VN_20241110220715.wav',
 '19_vi-VN_20241110220719.wav',
 '20_vi-VN_20241110220722.wav']

In [21]:
for wav_file in wav_files[0:3]:
    print(speech_recognition_from_file_with_endpoint(os.path.join(output_folder,wav_file), "vi-VN", endpoint_id))

Tôi có thể được trợ giúp bằng tiếng anh không?
Máy giặt của tôi không hoạt động, tôi phải làm gì?
Lg có chính sách bảo hành bao lâu?


## 3. Delete the all resources after testing
- Don't forget to delete the deployed custom model and project after testing.

In [23]:
delete_endpoint(base_url, headers, endpoint_id)

The endpoint deleted: 4b91679e-8605-4a30-8496-7b16df2df866, None


In [4]:
delete_project(base_url, headers, project_id)

The project deleted: 8b0eb25b-38f1-4f7b-8cfe-641753cd9cd1, None
